In [39]:
import numpy as np
import cvxpy as cp
def min_SINR(G,Pmax,Preceived,sigma,Group,Group_max,eps = 0.05):

  n,m = np.shape(G)
  p = cp.Variable((n,1))
  optimalPow = np.zeros(n)
  delta = np.identity(n)
  SignalMatrix = G*np.eye(n) 

  InterferenceMatrix = G-SignalMatrix 
  num_groups = int(np.size(Group,0))
  Grp_norm = np.array([group/np.sum(group) for group in Group])


  r = 1e4
  s = 0
  
  alpha = cp.Parameter()
  constraints = [p <= Pmax, p >= 0, Grp_norm*p <= Group_max, G*p <= Preceived, InterferenceMatrix@p + sigma <= alpha*SignalMatrix*p]

  obj = cp.Minimize(alpha)
  alpha.value = r
  prob = cp.Problem(obj, constraints)
  prob.solve()
  alpha.value = s
  prob = cp.Problem(obj, constraints)

  maxLoop = int(1e7)
  for i in range(1,maxLoop):
    alpha.value = (r + s)/2.0
    
    if r-s <= eps:
      break
    prob = cp.Problem(obj, constraints)
    prob.solve()
    if prob.status == 'optimal':
      r = alpha.value
      optimalPow = p.value
    else:
      s = alpha.value
    if r - s > eps and i == (maxLoop-1):
      print("not converge to eps")
  return l,u,alpha.value,optimalPow

In [40]:
G = np.array([[1.0,0.1,0.2,0.1,0.0],
                [0.1,1.0,0.1,0.1,0.0],
                [0.2,0.1,2.0,0.2,0.2],
                [0.1,0.1,0.2,1.0,0.1],
                [0.0,0.0,0.2,0.1,1.0]])

Grpmax = np.array([[3.0],[6.0]])
n,m = np.shape(G)
sigma = np.array([0.5,0.5,0.5,0.5,0.5]).reshape((n,1))
Group = np.array([[1.,1.,0,0,0],[0,0,1.,1.,1.]])


Preceived = (np.array([5.,5.,5.,5.,5.]).reshape((n,1)))/n

Pmax = np.array([3.]*n).reshape((n,1))


l,u,alp,best=min_SINR(G,Pmax,Preceived,sigma,Group,Grpmax)

print('Power=%s'%(best))
print('Min SINR=%.4g'%(1/alp))


Power=[[0.51316491]
 [0.49975582]
 [0.32181663]
 [0.54193416]
 [0.46441846]]
Min SINR=0.7598
